In [2]:
import pymongo
import pandas as pd
import jieba
import numpy as np
import string
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, Flatten
from keras.models import Sequential

In [3]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [4]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = doc.get("missionTags")
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pd.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

选调知多少政策宣讲活动                                                                                                           [就业发展, 宣讲]
会计专业课程组本科教学汇报                                                                                                      [教学建设, 报告/汇报]
喻园管理论坛: Understanding the Digital Resilience of Physicians during the COVID-19 Pandemic: An Empirical Study             [学术, 讲座]
国奖专访                                                                                                                  [采访, 校园生活]
AOL第三轮工作会议                                                                                                            [教学建设, 会议]
校运会跟拍                                                                                                                 [文体活动, 赛事]
学习贯彻党的二十大精神宣讲报告会                                                                                               [党团活动, 宣讲, 报告/汇报]
“廖建桥教室”揭牌仪式活动                                                                                    

In [5]:
# 忽略词
eng_words = list(string.ascii_lowercase + string.ascii_uppercase)
exclude_words = ['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&']
for w in eng_words:
    exclude_words.append(w)
for i in range(10):
    exclude_words.append(str(i))
print(exclude_words)

['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [6]:
# 分割 title 的词语
total_words = []
title_with_words = {}
for title in title_with_tags.keys():
    for i in exclude_words:
        title = title.replace(i, '')
    title_with_words[title] = []
    # jieba拆分方法
    words = jieba.cut(title, cut_all=False)
    for word in words:
        total_words.append(word)
        title_with_words[title].append(word)

fre = Counter(total_words)
word_fre = sorted(fre.items(), key=lambda i: i[1], reverse=True)
print("总词数: " + len(word_fre).__str__())
print(word_fre)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.643 seconds.
Prefix dict has been built successfully.


总词数: 79
[('管理', 7), ('论坛', 7), ('活动', 5), ('喻园', 5), ('宣讲', 3), ('二十大', 2), ('报告会', 2), ('仪式', 2), ('舍导', 2), ('交流', 2), ('学院', 2), ('第二届', 2), ('工商管理学', 2), ('科', 2), ('高端', 2), ('系列', 2), ('选调', 1), ('知多少', 1), ('政策', 1), ('会计专业', 1), ('课程', 1), ('组', 1), ('本科', 1), ('教学', 1), ('汇报', 1), ('国奖', 1), ('专访', 1), ('第三轮', 1), ('工作', 1), ('会议', 1), ('校运会', 1), ('跟', 1), ('拍', 1), ('学习', 1), ('贯彻', 1), ('党', 1), ('精神', 1), ('廖', 1), ('建桥', 1), ('教室', 1), ('揭牌', 1), ('领学团', 1), ('第一次', 1), ('集体', 1), ('备课', 1), ('会', 1), ('华工', 1), ('杯', 1), ('乒乓球赛', 1), ('半决赛', 1), ('热缩', 1), ('片', 1), ('发展', 1), ('对象', 1), ('开班', 1), ('杨院长', 1), ('教授', 1), ('党课', 1), ('十一月', 1), ('学术', 1), ('学科建设', 1), ('座谈会', 1), ('新进', 1), ('老师', 1), ('培训', 1), ('学生', 1), ('骨干', 1), ('训练营', 1), ('结营', 1), ('十二月', 1), ('中国', 1), ('管理学', 1), ('前沿', 1), ('研究', 1), ('研讨会', 1), ('开幕式', 1), ('与', 1), ('主', 1), ('报告', 1)]


In [8]:
# 查询 tag 的频次
def get_tag_times(dict_value_tags):
    tag_with_time = {}
    for key in dict_value_tags:
        for tag in dict_value_tags[key]:
            if tag not in tag_with_time:
                tag_with_time[tag] = 1
            else:
                tag_with_time[tag] += 1
    return tag_with_time


# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
print(len(tags_list))

16


In [9]:
# 构建词语数据矩阵
dist = np.zeros((len(title_with_tags), len(set(total_words))))
print(dist.shape)

(24, 79)


In [10]:
# 将数据填入矩阵
total_words_list = list(set(total_words))
title_list = list(title_with_words.keys())
for i in range(len(title_list)):
    print(title_with_words[title_list[i]])
    for word in title_with_words[title_list[i]]:
        for j in range(len(total_words_list)):
            if word == total_words_list[j]:
                print(word)
                dist[i][j] += 1

['选调', '知多少', '政策', '宣讲', '活动']
选调
知多少
政策
宣讲
活动
['会计专业', '课程', '组', '本科', '教学', '汇报']
会计专业
课程
组
本科
教学
汇报
['喻园', '管理', '论坛']
喻园
管理
论坛
['国奖', '专访']
国奖
专访
['第三轮', '工作', '会议']
第三轮
工作
会议
['校运会', '跟', '拍']
校运会
跟
拍
['学习', '贯彻', '党', '二十大', '精神', '宣讲', '报告会']
学习
贯彻
党
二十大
精神
宣讲
报告会
['廖', '建桥', '教室', '揭牌', '仪式', '活动']
廖
建桥
教室
揭牌
仪式
活动
['领学团', '第一次', '集体', '备课', '会']
领学团
第一次
集体
备课
会
['华工', '杯', '乒乓球赛', '半决赛']
华工
杯
乒乓球赛
半决赛
['热缩', '片', '活动']
热缩
片
活动
['发展', '对象', '开班', '杨院长', '教授', '党课']
发展
对象
开班
杨院长
教授
党课
['十一月', '舍导', '交流']
十一月
舍导
交流
['管理', '学院', '第二届', '工商管理学', '科', '高端', '论坛', '系列', '活动', '学术', '报告会']
管理
学院
第二届
工商管理学
科
高端
论坛
系列
活动
学术
报告会
['管理', '学院', '第二届', '工商管理学', '科', '高端', '论坛', '系列', '活动', '学科建设', '座谈会']
管理
学院
第二届
工商管理学
科
高端
论坛
系列
活动
学科建设
座谈会
['二十大', '宣讲']
二十大
宣讲
['新进', '老师', '培训']
新进
老师
培训
['学生', '骨干', '训练营', '结营', '仪式']
学生
骨干
训练营
结营
仪式
['十二月', '舍导', '交流']
十二月
舍导
交流
['中国', '管理学', '前沿', '研究', '研讨会', '开幕式', '与', '主', '报告']
中国
管理学
前沿
研究
研讨会
开幕式
与
主
报告


In [11]:
# 转化为 pd.DataFrame
data_words = pd.DataFrame(dist)
print(total_words_list)

['工商管理学', '半决赛', '学习', '会议', '管理', '校运会', '热缩', '知多少', '二十大', '工作', '领学团', '本科', '第一次', '学术', '十一月', '第三轮', '喻园', '汇报', '拍', '杨院长', '老师', '交流', '研究', '宣讲', '教授', '舍导', '精神', '培训', '党课', '研讨会', '报告会', '华工', '前沿', '片', '论坛', '骨干', '发展', '主', '十二月', '第二届', '仪式', '学院', '会', '建桥', '会计专业', '座谈会', '中国', '课程', '高端', '科', '贯彻', '政策', '开幕式', '学科建设', '教学', '廖', '与', '管理学', '国奖', '学生', '选调', '报告', '党', '备课', '揭牌', '结营', '杯', '集体', '教室', '系列', '活动', '乒乓球赛', '对象', '专访', '新进', '开班', '跟', '训练营', '组']


In [12]:
# tag 矩阵构建
# 定义 0 矩阵
dist_matrix = np.zeros((len(title_with_tags), len(get_tag_times(title_with_tags)),))
# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
# 获取 title 的 list
title_list = list(title_with_tags.keys())
# 根据 tag 生成矩阵
for mission_index in range(len(title_with_tags)):
    for tag in title_with_tags[title_list[mission_index]]:
        for tag_index in range(len(tags_list)):
            if tag == tags_list[tag_index]:
                dist_matrix[mission_index][tag_index] = 1

data_label = pd.DataFrame(dist_matrix)

In [13]:
# 分割数据为训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(data_words, data_label, test_size=0.2, random_state=0)

In [14]:
# 模型定义
def deep_model(feature_dim, label_dim):
    model = Sequential()
    print("create model. feature_dim ={}, label_dim ={}".format(feature_dim, label_dim))
    # model.add(Embedding(5, 32, input_length=79))
    # model.add(Flatten())
    model.add(Dense(512, activation='relu', input_dim=feature_dim))
    model.add(Dense(label_dim, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
# 模型训练函数
def train_deep(X_train, y_train, X_test, y_test, epochs):
    feature_dim = X_train.shape[1]
    label_dim = y_train.shape[1]
    model = deep_model(feature_dim, label_dim)
    model.summary()
    model.fit(X_train, y_train, batch_size=16, epochs=epochs, validation_data=(X_test, y_test))
    return model

In [16]:
# 开始训练
my_model = train_deep(X_train, y_train, X_test, y_test, 10)

create model. feature_dim =79, label_dim =16
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               40960     
                                                                 
 dense_1 (Dense)             (None, 16)                8208      
                                                                 
Total params: 49,168
Trainable params: 49,168
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2/2 [==============================] - 1s 203ms/step - loss: 0.6979 - accuracy: 0.0526 - val_loss: 0.6687 - val_accuracy: 0.2000
Epoch 2/10
2/2 [==============================] - 0s 39ms/step - loss: 0.6707 - accuracy: 0.0526 - val_loss: 0.6532 - val_accuracy: 0.4000
Epoch 3/10
2/2 [==============================] - 0s 37ms/step - loss: 0.6469 - accuracy: 0.1579 - val_loss: 0.6383 - val_ac